In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 4.8 MB/s 


## Import data (prot.fasta)

In [ ]:
fasta_file="/content/prot.fasta"

/content/prot.fasta


## Task 1

Let's remeber how to use dictionaries.

Task: return a dictionary where 
 * keys are IDs of seqs from an input fasta file (prot.fasta),
 * key's values are seqs itself. 

In [ ]:
def my_own_fasta_parser(inFile):
  sequences = {}
  with open (inFile) as handle:
      for f in handle:
          line = f.replace('\n','')
          if line.startswith(">"):
              seq_id = line[1:]
          else:
              sequences[seq_id]=line

  return sequences

my_own_fasta_parser(fasta_file)


{'seq0': 'FQTWEEFSRAAEKLYLADPMKVRVVLKYRHVDGNLCIKVTDDLVCLVYRTDQAQDVKKIEKF',
 'seq1': 'KYRTWEEFTRAAEKLYQADPMKVRVVLKYRHCDGNLCIKVTDDVVCLLYRTDQAQDVKKIEKFHSQLMRLME LKVTDNKECLKFKTDQAQEAKKMEKLNNIFFTLM',
 'seq10': 'FDSWDEFVSKSVELFRNHPDTTRYVVKYRHCEGKLVLKVTDNHECLKFKTDQAQDAKKMEK',
 'seq2': 'EEYQTWEEFARAAEKLYLTDPMKVRVVLKYRHCDGNLCMKVTDDAVCLQYKTDQAQDVKKVEKLHGK',
 'seq3': 'MYQVWEEFSRAVEKLYLTDPMKVRVVLKYRHCDGNLCIKVTDNSVCLQYKTDQAQDVK',
 'seq4': 'EEFSRAVEKLYLTDPMKVRVVLKYRHCDGNLCIKVTDNSVVSYEMRLFGVQKDNFALEHSLL',
 'seq5': 'SWEEFAKAAEVLYLEDPMKCRMCTKYRHVDHKLVVKLTDNHTVLKYVTDMAQDVKKIEKLTTLLMR',
 'seq6': 'FTNWEEFAKAAERLHSANPEKCRFVTKYNHTKGELVLKLTDDVVCLQYSTNQLQDVKKLEKLSSTLLRSI',
 'seq7': 'SWEEFVERSVQLFRGDPNATRYVMKYRHCEGKLVLKVTDDRECLKFKTDQAQDAKKMEKLNNIFF',
 'seq8': 'SWDEFVDRSVQLFRADPESTRYVMKYRHCDGKLVLKVTDNKECLKFKTDQAQEAKKMEKLNNIFFTLM',
 'seq9': 'KNWEDFEIAAENMYMANPQNCRYTMKYVHSKGHILLKMSDNVKCVQYRAENMPDLKK'}

## Task 2

Another super easy task.

We have the same fasta file (prot.fasta).

Now we want to get a list with the ids of protein seqs that have 
a relative frequency higher than a given threshold for a given residue.

And don't forget to use my_own_fasta_parser function from a previous task!

In [ ]:
def my_own_residue_abundance(input_file, residue, threshold=0.2):
    
    seq_id = []
    sequences = my_own_fasta_parser(input_file)

    for seq_id, sequence in sequences.items():
        #<<<<<<<<<<...>>>>>>>>>>>
        #<<<<<<<<<<...>>>>>>>>>>>
            #<<<<<<<<<<...>>>>>>>>>>>

    return seq_id

In [ ]:
def my_own_residue_abundance(input_file, residue, threshold=0.2):   
    sequence_id = []
    sequences = my_own_fasta_parser(input_file)

    for seq_id, sequence in sequences.items():
        frequency=(sequence.count(residue))/len(sequence)
        if frequency > threshold:
          sequence_id.append(seq_id)
    return sequence_id

In [ ]:
print(my_own_residue_abundance(fasta_file, 'D', threshold=0.08))

['seq0', 'seq2', 'seq3', 'seq8', 'seq10']


In [ ]:
print(my_own_residue_abundance(fasta_file, 'Q', threshold=0.05))

['seq1', 'seq2', 'seq3']


## Task 3

Let's practice in Bio package using.

Task:
1. read fasta file containing several DNA seqs (nucl.fasta)
2. subset seqs that have GC content > 45 and coding protein with aromaticity > 0.01
3. write a new fasta file with those protein(!) seqs
4. return the percentage of seqs that passed your filter
Hint: Bio.SeqIO, Bio.SeqRecord, Bio.SeqUtils

In [ ]:
import Bio
from Bio import SeqIO
from Bio import SeqRecord
from Bio import SeqUtils

In [ ]:
fasta_file1="/content/nucl.fasta"

In [ ]:
from Bio.SeqUtils import GC
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq


def my_own_filtering(input_file, output_file, filt_gc=45, filt_arom=0.125):
  sequences = {}
  c = 0 
  with open(input_file, "r") as content:   
      for record in SeqIO.parse(content, "fasta"):
        # calculate GC content using Bio
        c+=1
        calc_gc=GC(record.seq)
        # calculate aromaticity using Bio
        protein=str((record.seq).translate())
        line=ProteinAnalysis(protein)
        calc_arom=line.aromaticity()
        if calc_gc >= filt_gc and calc_arom >= filt_arom:
          sequences[record.id]=protein
  records = []
  for seq_id, seq in sequences.items():
    my_record=SeqRecord(Seq(seq), id=seq_id, name=seq_id, description=seq_id)
    records.append(my_record)
  SeqIO.write(records,output_file, "fasta")
  return(round(float(len(sequences)/c)*100))


In [ ]:
my_own_filtering(fasta_file1, "/content/check1.fasta", filt_gc=45, filt_arom=0.125)

/usr/local/lib/python3.7/dist-packages/Bio/Seq.py:2983: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


14

In [ ]:
for seq_record in SeqIO.parse("/content/check1.fasta","fasta"):
    print (seq_record)

ID: id5
Name: id5
Description: id5
Number of features: 0
Seq('ISFARPRH')


## Task 4

Continue practicing in Bio package using:

Task:

Complete the following code that should be able to return 
the best alignment of two amino acid seqs (pairwise2.align.globalds) based on BLOSUM62 matrix from Bio.SubsMat.MatrixInfo.
http://rosalind.info/glossary/blosum62/

In [ ]:
from Bio import pairwise2
from Bio.SubsMat import MatrixInfo as matlist
def balign(first_seq, second_seq, gap_open, gap_extend):
  # Load the matrix
  matrix = matlist.blosum62
  # Generate the alignments
  alns = pairwise2.align.globalds(first_seq,second_seq,matrix, gap_open, gap_extend)
  # Extract the best alignment (first one in the alns list)
  top_aln = alns[0]
  # Print the alignment
  aln_A, aln_B, score, begin, end = top_aln.seqA,top_aln.seqB,top_aln.score,top_aln.start,top_aln.end
  return(f"Best alignment: seq1 = {aln_A}, seq2 = { aln_B}, score = {round(score)}, begin = {begin},end = {end}")




In [ ]:
first_seq="MEEPQSDPSVEPPLSQETFSD"
second_seq="MEESQSDISLELPLSQETFSGLW"
gap_open=-1
gap_extend=-0.5

balign(first_seq, second_seq, gap_open, gap_extend)


'Best alignment: seq1 = MEEPQSDP-SVEP-PLSQETFSD--, seq2 = MEESQSD-ISLE-LPLSQETFSGLW, score = 72, begin = 0,end = 25'

## Task 5

You have some DNA sequence: *AGTACTAGAGCATTCTATGGAG*.
Find out which peptides could be created from it and sort them by their length.
Use as much Biopython modules as possible.

In [ ]:

from Bio.Seq import Seq
from Bio.Data import CodonTable

standard_table = CodonTable.unambiguous_dna_by_name["Standard"]
protein_pep = []
seq = Seq("AGTACTAGAGCATTCTATGGAG")

# start from different nucleotide
for i in range (0,len(seq)-2):
  peptide=seq[i:].translate(standard_table)
  protein_pep.append(peptide)
# end at different nucleotide
for j in range (3,len(seq)):
  peptide=seq[:j].translate(standard_table)
  protein_pep.append(peptide)

protein_pep = list(set(protein_pep))
protein_pep.sort(key=len, reverse=True)
print(protein_pep)


[Seq('STRAFYG'), Seq('VLEHSME'), Seq('Y*SILW'), Seq('STRAFY'), Seq('TRAFYG'), Seq('LEHSME'), Seq('STRAF'), Seq('EHSME'), Seq('*SILW'), Seq('RAFYG'), Seq('AFYG'), Seq('HSME'), Seq('SILW'), Seq('STRA'), Seq('STR'), Seq('ILW'), Seq('FYG'), Seq('SME'), Seq('ST'), Seq('ME'), Seq('YG'), Seq('LW'), Seq('W'), Seq('E'), Seq('S'), Seq('G')]


/usr/local/lib/python3.7/dist-packages/Bio/Seq.py:2983: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


Second option



In [ ]:
from Bio.Seq import Seq
from Bio.Data import CodonTable

standard_table = CodonTable.unambiguous_dna_by_name["Standard"]
protein_pep = []
seq = Seq("AGTACTAGAGCATTCTATGGAG")

# start from different nucleotide
for i in range (0,len(seq)-2):
  for j in range (2,len(seq)):
    peptide=seq[i:j].translate(standard_table)
    protein_pep.append(peptide)

protein_pep = list(set(protein_pep))
protein_pep.sort(key=len, reverse=True)
print(protein_pep)


[Seq('STRAFYG'), Seq('Y*SILW'), Seq('TRAFYG'), Seq('STRAFY'), Seq('VLEHSM'), Seq('STRAF'), Seq('TRAFY'), Seq('*SILW'), Seq('VLEHS'), Seq('Y*SIL'), Seq('RAFYG'), Seq('LEHSM'), Seq('VLEH'), Seq('LEHS'), Seq('RAFY'), Seq('AFYG'), Seq('SILW'), Seq('STRA'), Seq('Y*SI'), Seq('EHSM'), Seq('*SIL'), Seq('TRAF'), Seq('TRA'), Seq('Y*S'), Seq('STR'), Seq('AFY'), Seq('SIL'), Seq('FYG'), Seq('RAF'), Seq('EHS'), Seq('*SI'), Seq('LEH'), Seq('ILW'), Seq('HSM'), Seq('VLE'), Seq('RA'), Seq('HS'), Seq('LE'), Seq('IL'), Seq('SI'), Seq('YG'), Seq('SM'), Seq('VL'), Seq('LW'), Seq('Y*'), Seq('EH'), Seq('*S'), Seq('TR'), Seq('FY'), Seq('ST'), Seq('AF'), Seq('G'), Seq('I'), Seq('*'), Seq('W'), Seq('S'), Seq('T'), Seq('Y'), Seq('A'), Seq('L'), Seq('R'), Seq('F'), Seq('E'), Seq('V'), Seq('H'), Seq('M'), Seq('')]
67


/usr/local/lib/python3.7/dist-packages/Bio/Seq.py:2983: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


## Task 6

TASK: Try to create one-line function (without (!!!) using Bio package)
that returns reverse complementary to a given sequence. 
Hint: using dictionary & list comprehensions might be helpful.

In [ ]:
# for DNA
def rev_compl_one_line(seq):
    return (''.join(([{'A':'T','T':'A','C':'G','G':'C'}.get(x) for x in seq[::-1]])))

In [ ]:
# for RNA
def rev_compl_one_line_rna(seq):
    return (''.join(([{'A':'U','U':'A','C':'G','G':'C'}.get(x) for x in seq[::-1]])))

In [ ]:
#DNA
seq='GAAACCCTTAC'
rev_compl_one_line(seq)

'GTAAGGGTTTC'

In [ ]:
#RNA
rna_seq='GAAACCCUUAC'
rev_compl_one_line_rna(rna_seq)

'GUAAGGGUUUC'